In [ ]:
import json
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime

# 定义生成车辆数据的函数
def generate_vehicle_data(O, D, battery, start_charge, energy_consumption, start_date, od_cnt):
    # 初始化一个空列表，用于存储每辆车的数据
    vehicles = []
    
    # 遍历每辆车，生成对应属性
    for i in range(od_cnt):
        # 从正态分布中抽样生成 battery 和 start_charge
        battery_sample = np.random.normal(loc=battery, scale=5)
        start_charge_sample = np.random.normal(loc=start_charge, scale=5)
        
        # 从正态分布中抽样生成 energy_consumption，标准差为2
        energy_consumption_sample = np.random.normal(loc=energy_consumption, scale=2)
        
        # 创建一个字典来存储每辆车的数据
        vehicle_data = {
            "O": O,
            "D": D,
            "battery": round(battery_sample, 2),  # 保留两位小数
            "start_charge": round(start_charge_sample, 2),  # 保留两位小数
            "energy_consumption": round(energy_consumption_sample, 2),  # 保留两位小数
            "start_date": start_date
        }
        
        # 将字典添加到车辆列表中
        vehicles.append(vehicle_data)
    
    # 将数据转换为JSON格式
    json_data = json.dumps(vehicles, indent=4)
    
    # 保存到文件
    with open("vehicle_data.json", "w") as f:
        f.write(json_data)
    
    # 返回JSON数据字符串
    return json_data

# 示例调用函数
o = "Point A"
d = "Point B"
battery = 80
start_charge = 50
energy_consumption = 10
start_date = "2024-10-23"
od_cnt = 5

# 生成车辆数据并输出
vehicle_json = generate_vehicle_data(o, d, battery, start_charge, energy_consumption, start_date, od_cnt)
print(vehicle_json)

# 读取生成的JSON文件并转换为DataFrame
def read_vehicle_data_to_dataframe(file_path):
    with open(file_path, "r") as f:
        vehicle_data = json.load(f)
    
    # 将JSON数据转换为DataFrame
    df = pd.DataFrame(vehicle_data)
    return df

# 示例读取数据并转换为DataFrame
dataframe = read_vehicle_data_to_dataframe("vehicle_data.json")
print(dataframe)

# 读取当前文件夹下所有文件夹名并处理CSV文件
def process_folder_structure():
    current_dir = os.getcwd()
    folders = [f for f in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, f))]
    
    for folder in folders:
        sub_folder_path = os.path.join(current_dir, folder)
        sub_folders = [sf for sf in os.listdir(sub_folder_path) if os.path.isdir(os.path.join(sub_folder_path, sf))]
        
        for sub_folder in sub_folders:
            final_path = os.path.join(sub_folder_path, sub_folder)
            car_vis_path = os.path.join(final_path, 'car_vis.csv')
            station_vis_path = os.path.join(final_path, 'station_vis.csv')
            
            if os.path.exists(car_vis_path):
                car_vis_df = pd.read_csv(car_vis_path)
                car_vis_df['geometry'] = car_vis_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
                car_gdf = gpd.GeoDataFrame(car_vis_df, geometry='geometry')
                
                # 读取面要素文件，假设面要素文件路径为polygon_shapefile_path
                polygon_shapefile_path = "polygon_shapefile.shp"
                if os.path.exists(polygon_shapefile_path):
                    polygons = gpd.read_file(polygon_shapefile_path)
                    filtered_car_gdf = gpd.sjoin(car_gdf, polygons, op='intersects')
                    filtered_car_gdf.to_file(os.path.join(final_path, 'filtered_car_vis.shp'))
            
            if os.path.exists(station_vis_path):
                station_vis_df = pd.read_csv(station_vis_path)
                station_vis_df['geometry'] = station_vis_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
                station_gdf = gpd.GeoDataFrame(station_vis_df, geometry='geometry')
                
                # 读取面要素文件，假设面要素文件路径为polygon_shapefile_path
                if os.path.exists(polygon_shapefile_path):
                    polygons = gpd.read_file(polygon_shapefile_path)
                    filtered_station_gdf = gpd.sjoin(station_gdf, polygons, op='intersects')
                    filtered_station_gdf.to_file(os.path.join(final_path, 'filtered_station_vis.shp'))

# 示例调用函数处理文件夹结构
process_folder_structure()
